### Automated Concept Extraction

There are several steps to the automated concept extraction method that are outlined in their paper [here]().

Firstly, we need to create patches from images that represent the object we want to derive concepts for. This involves using skimage segmentation and extracting the patches and superpixels from this. These will be used to find visual features that can be used as concepts.

Once we have the superpixels that we need, we can make use of a clustering technique on the representations extracted from the bottleneck layers of our model after passing a superpixel through. This will allow us to find visually similar images that will hopefully group patches that represent the same visual concept.

These groups of superpixels can then be used to create a concept activation vector. This involves getting the activations of these superpixels relating to a concept and then random superpixels that as a group represent no descernable concept. A linear classifier is trained on these examples and the vector orthognal to the hyperplane that separates the concept examples from the random is taken to be the concept activation vector.

The influence of this concept can be determined by taking the partial derivative of the class logit you want to examine with respect to a bottleneck layer. Multiplying the CAV by this partial derivative will allow us to determine the impact this concept had on the prediction.

This concludes the rough overview of the method that will be employed. The aim is to create realistic, reasonable concepts from the mitotic figures without the need of manually gathering images of specific concepts.

### Importing libraries

We will be making use of the following libraries to implement our method.

In [1]:
import sys
import random
from pathlib import Path
import numpy as np
import sklearn.metrics as metrics
from PIL import Image

import Utils.ACE.ace_helpers as ace_helpers
from Utils.ACE.ace import ConceptDiscovery
from Utils.ACE.run_ace import run_concept_discovery

### Testing with COCO

We will begin by taking some images from the COCO dataset, specifically those inclusing a tennis racket. We will use this example to test our method and ensure we are processing the images correctly and the results seem reasonable. This seems the best course of action as I have a better understanding of the visual features that concern a tennis racket and no formal understanding of the visual features of a mitotic figure.

I believe that using the COCO images may result in poor concepts as the images are not specifically tennis rackets, but instead images that contain tennis rackets. This means that we will get many superpixels that are not relevant to tennis rackets.

In [2]:
# Create an output directory for our data
output = Path.cwd() / "ACE_COCO_output/"

# Create the sub directories at the output location.
ace_helpers.create_directories(output, remove_old=False)

In [3]:
# Specify the target class and the source directory.
# Note: The target class should be a folder in the source directory.
# This folder should have a folder called discovery for images for concept discovery and a folder called tcav for tcav score calculation.
target_class = "tennis racket"
source_dir = "D:\DS\DS4\Project\COCO"

We have defined the output path and the source directory where our images for discovery and tcav scores are stored.

Now we need to load in our model, this is wrapped in the ModelWrapper class which will extract the activations and gradients for us so we can cluster the superpixels, create cavs adn get tcav scores.

### Selecting the bottleneck layers

In order to extract the activations and gradients from a layer, we need to determine which layer(s) are bottleneck layers. A bottleneck layer typically reduces the number of channels in the data between the input and output while keeping the size of the image equal by using a kernel of (1,1) and a stride of (1,1). This means that the model compresses the representation of the input in this layer and keeps the most important features for performing the task. This makes it the ideal layer for using the activations from to cluster the superpixels for ACE and to train the linear classifier to find a CAV.

Looking at the model structure below we can see that there are several such layers in the backbone of our model. It may be worth just taking a selection of these. I have decided to take the bottleneck from the last bottleneck unit in each layer. This means I will be using the following 4 layers.

```
bottleneck_layers = ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1']
```

These will be the layers I extract both the activations from and the gradients when looking at the influence of each concept.

In [4]:
# Create a list of the bottleneck layers.
bottleneck_layers = ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1']

# Create the model variable and set it to evaluate.
mymodel = ace_helpers.MyModel("tmp", bottleneck_layers)
mymodel.model.eval()
mymodel.model.model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

### ConceptDiscovery class

Now that we have all of the required parameters, we can initialize the ConceptDiscovery class, which contains the methods for creating superpixels, clustering to find concepts, creating concept activation vectors and testing these. We will make use of this class for most of the notebook.

In [5]:
# Creating the ConceptDiscovery class instance.
cd = ConceptDiscovery(
    mymodel,
    target_class,
    source_dir,
    output,
    bottleneck_layers,
    num_random_exp=2,
    channel_mean=True,
    max_imgs=10,
    min_imgs=5,
    num_discovery_imgs=10)

Now we can call create_patches. This function takes the discovery images in the source directory and segments the image into superpixels for use later. The number of segments can be specified below.

These superpixels are then saved in the concept subdirectory of the output along with the patches and the discovery images.

In [ ]:
# Creating the dataset of image patches.
cd.create_patches(param_dict={'n_segments': [15]})

# Saving the concept discovery target class images.
image_dir = cd.discovered_concepts_dir / 'images'
image_dir.mkdir()
ace_helpers.save_images(image_dir.absolute(),
                        (cd.discovery_images * 256).astype(np.uint8))

The discover_concepts function takes all of the superpixels that were produced and determines possible concepts. This is done by passing the superpixels through the model and extracting the activations in the bottleneck layers. These representations of the superpixels can be clustered to find similar superpixels, which can be considered to be a possible visual concept.

In [ ]:
# Discovering Concepts
cd.discover_concepts(method='KM', param_dicts={'n_clusters': 25})

Now we can save these discovered concepts to the concept folder for visual review and use later on when generating concept activation vectors.

In [ ]:
# Save discovered concept images (resized and original sized)
ace_helpers.save_concepts(cd)

In order to test the validity of our findings, it is important to compare our CAV against a random concept. In addition, I will carry out a two-sided t-test in order to statistically quantify that the resultant scores are significantly different from one another. 

The initialize_random_concept_and_samples method creates a folder called Random in the concept directory and randomly samples superpixels for a random concept and a number of random samples for use as a random counterpart for CAV generation. This will allow us to determine that the found CAV from our clustered superpixels can explain the prediction better than a random selection, helping to qualify the findings.

In [ ]:
cd.initialize_random_concept_and_samples()

The cavs method takes the superpixels from each of the concept directories and a random counterpart and finds the resultant CAV. A CAV will be generated for every pair of concept and random counterpart. If we are carrying out N experiments and we have M concepts discovered we will have N * (M + 1) CAVs, as we have to include the random concept with the M found potential concepts from clustering.

In [ ]:
cav_accuracies = cd.cavs()

We can see the returned accuracies for each CAV generated (The experiments are listed). This helps us to understand how well the linear classifier splits the potential concepts superpixels from the randomly selected superpixels.

In [ ]:
cav_accuracies

Now we can get a score of how influencial the individual CAVs are on all of the predictions on images from the tcav folder within the source directory.

This tcavs method takes the images from the tcav folder and calculates the gradients for the class of interest with respect to the bottleneck layers. These gradients are then multiplied by the CAV vector to determine how relevant the CAV is in the prediction. This is results in a list of scores for each concept, an entry for each random counterpart/ experiment.

These scores are then used in a two-sided t-test with the random concept to determine if the potential concept is statistically different than a random selection of the superpixels. This will aid in validiating any findings from the method.

In [ ]:
scores = cd.tcavs(test=False, sort=False)

In [ ]:
scores

The CAV accuracies and the TCAV scores can be saved into a report text file for review. This includes the average accuracy for a CAV, average tcav score for a CAV and the p-value from the t-test.

In [ ]:
cd.save_ace_report()

We can look at the superpixels that make up a concept to visually identify any characteristics that may have been used to cluster these superpixels together to find a potential concept.

In [6]:
# Plot examples of discovered concepts
for bn in cd.bottlenecks:
    cd.plot_concepts(bn, 10)

This concludes our test implementation of the method. We will now sanity check several components of the code to ensure that there are no issues and components are working as expected.

### Sanity check

To ensure that the implementation is working as I expect it to I will check the following:

That the activations are equivalent both individually and in batches.

To check this I will manually run the activations both individually and in a batch and ensure the result is the same.

In [ ]:
# Load in some test files for passing to get activations
test_files = np.array(list((cd.discovered_concepts_dir / "superpixels").iterdir())[:2])

In [ ]:
# Get the activations with a batch size of 1.
acts = cd._get_activations(test_files, bs=1)

In [ ]:
# Get the gradients with a batch size of 2.
batch_acts = cd._get_activations(test_files, bs=2)

In [ ]:
# Check that the activations are equal.
np.unique(acts["backbone.body.layer1.2.conv1"] == batch_acts["backbone.body.layer1.2.conv1"])

We can see that the activations are equivalent, so we have checked this.

Next, I want to check that the hooks don't require the gradients to be zeroed and are not being accumulated.

This will involve getting the returned gradients, and ensuring that every element in the list is not smaller than the next element in the list. If the gradients were accummulating, this would be the case.

In [ ]:
# Get the gradients for our test images.
grads, _ = cd._return_gradients(test_files, test=False)

In [ ]:
# Check that every value in gradient i-1 in the list is not smaller than in gradient i.
layer_grads = grads["backbone.body.layer1.2.conv1"]
for i in range(1, len(layer_grads)):
    print(np.unique(layer_grads[i - 1] <= layer_grads[i]))

We can see that there are instances where this is false, so the gradient in gradient i-1 is greater than in gradient i at some values. This shows that the gradients are not being accummulated.

Lastly, I want to ensure that the method of calculating the TCAV score is working as expected and producing correct scores. This will involve manually running the method and ensuring the output is correct.

In [ ]:
# Get some gradients to use
gradients, _ = cd._return_gradients(test_files)

In [ ]:
cd._tcav_score("backbone.body.layer1.2.conv1", "tennis racket_concept1", "Random_001", gradients)

In [ ]:
# Get the CAV vector.
tmp_vector = cd.load_cav_direction("tennis racket_concept1", "Random_001", "backbone.body.layer1.2.conv1")

# Check the shape of the vector.
tmp_vector.shape

In [ ]:
# Get these test gradients.
test_grads = gradients["backbone.body.layer1.2.conv1"]

# Check the test gradients shape.
len(test_grads), len(test_grads[0])

In [ ]:
# Multiply the vector against the gradient.
prod = (test_grads * vector)
prod.shape

In [ ]:
# Get the sum along axis 1 so we have the dot products.
dot_products = np.sum(prod, -1)

In [ ]:
# Get the average value of the booleans returned.
np.mean(dot_products < 0)

We can see that the resultant score is identical for both. The manual method I have used follows the original implementation of the tcav score in the paper.

### ACE on Mitotic figures



In [ ]:
# Create an output directory for our data
output = Path.cwd() / "ACE_mitotic_output/"

target_class = "mitotic figure"
source_dir = "D:\DS\DS4\Project\MIDOG"

bottleneck_layers = ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1']

model_params = {"model": "mitotic", "layers": bottleneck_layers}

cd_params = {"bottlenecks": bottleneck_layers, "num_random_exp": 100, "channel_mean": True, "max_imgs": 40, "min_imgs": 20, "num_discovery_imgs": 40, "average_image_value": 117, "resize_dims": (512, 512)}

patch_params = {"method": 'slic', "param_dict": {'n_segments': [15]}}

clustering_params = {"method": 'KM', "param_dicts": {'n_clusters': 25}}

cav_params = {"min_acc": 0.5}

tcav_params = {"test": False, "sort": True}

In [ ]:
run_concept_discovery(output, target_class, source_dir, model_params, cd_params, patch_params, clustering_params, cav_params, tcav_params)
     

In [ ]:
# Creating the ConceptDiscovery class instance.
cd = ConceptDiscovery(
    mymodel,
    target_class,
    random_concept,
    ['backbone.body.layer1.2.conv1', 'backbone.body.layer2.3.conv1', 'backbone.body.layer3.5.conv1', 'backbone.body.layer4.2.conv1'],
    source_dir,
    activations_dir,
    cavs_dir,
    num_random_exp=2,
    channel_mean=True,
    max_imgs=100,
    min_imgs=50,
    num_discovery_imgs=100,
    num_workers=0)

In [ ]:
# Creating the dataset of image patches.
cd.create_patches(discovered_concepts_dir, param_dict={'n_segments': [15]})

# Saving the concept discovery target class images.
image_dir = discovered_concepts_dir / 'images'
image_dir.mkdir()
ace_helpers.save_images(image_dir.absolute(),
                        (cd.discovery_images * 256).astype(np.uint8))

In [ ]:
# Discovering Concepts
cd.discover_concepts(discovered_concepts_dir, method='KM', param_dicts={'n_clusters': 25})

In [ ]:
# Save discovered concept images (resized and original sized)
ace_helpers.save_concepts(cd, discovered_concepts_dir)

In [ ]:
# TODO add to helper function generate_random
superpixels = discovered_concepts_dir / "superpixels"
list_of_files = list(superpixels.iterdir())

# Random selection of the the superpixels for random concept?
random.seed(42)
cd.random_imgs = np.array(random.sample(list_of_files, 50))

In [ ]:
# Save the random imgs for review save_random
for img in cd.random_imgs:
    destination = img.parent.parent / "Random"
    destination.mkdir(exist_ok=True)
    
    shutil.copy(img, destination / img.name)

In [ ]:
cav_accuracies = cd.cavs()